<h1>Data Collection Website - created July 2020</h1>

This is a test web app that I have created which:
    
    (i) takes input from a user (email address and height data)
    (ii) uploads the data to an sql database hosted on heroku
    (iii) quiries the database to find the average height value
    (iv) sends an email to the user with the average height value from the sql database
    
I am using flash for rendering the website and sqlalchemy to quiry the sql database

The webpages are named:

    (i) templates/index.html (main page) and 
    (ii) templates/success.html (where the user is sent to when data has been successfully entered). 
    The css stylesheet is called static/main.css
    
Please note that the code below has had the username/password for google email disabled

The working app can be seen and used at:
    bournedc.herokuapp.com
    
Mike Bourne

In [ ]:
from flask import Flask,render_template,request
from flask_sqlalchemy import SQLAlchemy 
from sqlalchemy.sql import func

#Initial the Flask appllication called 'app'
app=Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI']=  'postgres://xxxxxxxx'


#Initiate the SQL database called 'db'
db = SQLAlchemy(app)


# The following Data class defines an object which consists of an email address and a height
class Data(db.Model):
    __tablename__ = "data"
    id=db.Column(db.Integer, primary_key=True)
    email=db.Column(db.String(120), unique=True)
    height=db.Column(db.Integer)
 
    #initiating a Data object 
    def __init__(self,email,height):
        self.email = email
        self.height = height

        
#This function sends the email to the user using the smtplib library. 
#The function requires an email address 'to' and 
#two integer values 'height' and average'height'

def send_email(to,height, average_height):
    import smtplib
    from email.mime.text import MIMEText

    #This is the email address from which an email will addressed from (disabled)
    fromMy = 'xxxxxxx@gmail.com'

    subject="Mike Bourne's Height App"
    body = """Thank-you for using the app. 
    You entered the following height info: %s \n
    The average height of all the people who have entered data is: %s \n
    \n
    Thank-you, Mike""" % (height, average_height)

    message = """From: %s\nTo: %s\nSubject: %s\n\n%s
    """ % (fromMy, to, subject, body)
 
    #These are the login details for the google email account being used (disabled)
    username = 'xxxxxxxx@gmail.com'
    password = 'xxxxxxxxxx'

    #These commands, using the smtplib account, 
    # are used to login into gmail and send the email
    server = smtplib.SMTP("smtp.gmail.com",587)
    server.ehlo()
    server.starttls()
    server.login(username, password)
    server.sendmail(fromMy, to,message)
    server.quit()    


#defining the main starting page of the webapp - 'index.html'
@app.route("/")
def index():
    return render_template("index.html")

#defining the second page of the webapp - 'success.html'
#This is where the user is sent when thy click SUBMIT on the index.html page
@app.route("/success", methods=['POST'])
def success():    
    if request.method == 'POST':
        #gather the data from the index.html page. Asign to vars email and height
        email = request.form['email_name']
        height= request.form['height_name']
        #Check if email address is already in the database
        if db.session.query(Data).filter(Data.email == email).count() == 0:
            #if a new address, create an instance of the Data 
            # and add this data to the sql database
            data=Data(email,height)
            db.session.add(data)
            db.session.commit()
            #quiry the database to find average height in database
            aver_height = db.session.query(func.avg(Data.height)).scalar()
            #round value to one decimal point
            aver_height=round(aver_height,1)
            #call the send_email to send an email to the user
            send_email(email,height,aver_height)
            #change the webpage to 'success.html'
            return render_template("success.html")
    #if the email address has already been used (i.e. db quiry > 0)
    # then inform user that there is an error
    return render_template("index.html", 
                text_box="this email address has been used - no data uploaded")


if __name__ == '__main__':
    app.debug = True
    app.run()

The output email reads:
    
    Thank-you for using the app. 
    You entered the following height info: 178 

    The average height of all the people who have entered data is: 179.0 

    

    Thank-you, Mike